### keras-retinanet 패키지를 이용하여 이미지와 영상 Object Detection 수행
*  Pretrained된 coco 모델을 로드 하고 이를 이용하여 Object Detection 수행

#### 관련 모듈 import 

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
%matplotlib inline


# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu

#### gpu 환경 설정

In [ ]:
# 현 버전에서 이슈가 있어서 주석 처리합니다. 2020.08.29
#gpu = 0
#setup_gpu(gpu)

#### keras-retinanet으로 pretrained된 coco 모델 다운로드하고 해당 모델을 로드

In [ ]:
# 아래 모델은 https://github.com/fizyr/keras-retinanet/releases 에서 download 받을 수 있음. 
# 해당 모델 h5 파일을 snapshot 디렉토리에 저장 후 retina model의 load_model()을 이용하여 모델 로딩.
# !wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5 

In [ ]:
os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

In [ ]:
model_path = os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

# pretrained coco 모델 파일을 retinanet 모델로 로딩.  
retina_model = models.load_model(model_path, backbone_name='resnet50')

#### coco 데이터 세트의 클래스id별 클래스명 지정. 

In [ ]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' }

In [ ]:
test_array = np.array([[1,2]])
print(test_array.shape)
expand_array = np.expand_dims(test_array, axis=0)
expand_array.shape

#### 원본 이미지를 Object Detection 후에 시각화 표시. 

In [ ]:
# load image
image = read_image_bgr('../../data/image/beatles01.jpg')
print('image shape:', image.shape)
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)


# 모델에 입력전에 이미지 사전 처리. keras-retinanet은 image
image = preprocess_image(image)
image, scale = resize_image(image)
print('resized image size:', image.shape, 'scale:', scale)

# 이미지에 대해 Object Detection 수행. 
start = time.time()
boxes, scores, labels = retina_model.predict_on_batch(np.expand_dims(image, axis=0))
print(boxes.shape, scores.shape, labels.shape)
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(14, 14))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
def get_detected_image_retina(model, img_array, use_copied_array, is_print=True):
    
    # copy to draw on
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    img_array = preprocess_image(img_array)
    img_array, scale = resize_image(img_array)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img_array, axis=0))
    if is_print:
        print("object detection 처리 시간: ", round(time.time() - start,5))
    
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw_img, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
        draw_caption(draw_img, b, caption)
    
    if is_print:
        print("이미지 processing 시간: ", round(time.time() - start,5))
    
    return draw_img

# image 로드 
img = cv2.imread('../../data/image/john_wick01.jpg')
draw_img = get_detected_image_retina(retina_model, img, use_copied_array=True, is_print=True)
img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(14, 14))
plt.imshow(img_rgb)

In [ ]:
def detect_video_retina(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size= (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)
    
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break

        detected_image = get_detected_image_retina(model,image_frame, use_copied_array=False, is_print=True)
        vid_writer.write(detected_image)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
detect_video_retina(retina_model, '../../data/video/John_Wick_small.mp4', output_path="../../data/output/John_Wick_small_retina01.avi")

In [ ]:
!gsutil cp ../../data/output/John_Wick_small_retina01.avi gs://my_bucket_dlcv/data/output/John_Wick_small_retina01.avi

In [ ]:
detect_video_retina(retina_model, '../../data/video/Night_Day_Chase.mp4', output_path="../../data/output/Night_Day_Chase_retina01.avi")

In [ ]:
!gsutil cp ../../data/output/Night_Day_Chase_retina01.avi gs://my_bucket_dlcv/data/output/Night_Day_Chase_retina01.avi